# NeuroMANCER dynamical systems, system identification


## NeuroMANCER and Dependencies

### Install (Colab only)
Skip this step when running locally.

In [ ]:
!pip install "neuromancer[examples] @ git+https://github.com/pnnl/neuromancer.git@master"

# Input signals in neuromancer.psl

For a system identification task, a sequences of input signals must be generated in order to perturb the system across the dyanmic range of the state space. Neuromancer.psl has functions to generate all the standard perturbations used in system identification including:

+ Random walk: walk
+ White noise: noise
+ Random step functions: step
+ Randomly offset periodic functions (sin, cos, square, sawtooth): periodic
+ Sum of sines: sines
+ Auto-regressive moving average: arma
+ Cubic spline interpolation between randomly sampled points: splines

All input signal generator functions have the arguments: 

+ nsim: Length of input signal time series
+ d: Dimension of input signal time series
+ min: Minimum value for input signal time series
+ max: Maximum value for input signal time series

All signals are by default scaled to between min and max, however the unbounded signals white noise, random walk, and ARMA
have a bounded argument which can be set to false if an unbounded signal is desired. 

Below are plots of each signal time series generated with nsim=1000 and d=3.

In [5]:
from neuromancer.psl.signals import signals
import matplotlib.pyplot as plt
%matplotlib inline

ModuleNotFoundError: No module named 'functorch'

In [ ]:
signals

In [ ]:
for n, s in signals.items():
    x = s(1000, 3, min=-2., max=1.5)
    print(n, x.shape)
    plt.plot(x)
    plt.show()
    plt.close()

# Systems in NM's Python Systems Library (psl)

The psl subpackage provides systems models for a range of autonomous, nonautonomous, networked, and building envelope systems, as well as an interface for loading and interacting with pre-recorded measurement datasets in common with the systems models. 

In [ ]:
import neuromancer
neuromancer.psl.systems

In [ ]:
for n, sys in neuromancer.psl.systems.items():
    print(n)
    sys().show()
    plt.close()

In [ ]:
sys = neuromancer.psl.systems['CSTR']()
print(f'Changing inputs and states: {sys.variables}')
print(f'Constants defining simulation: {sys.constants}')
print(f'Parameters of the system: {sys._params}')
print(f'number of inputs: {sys.nu}')
print(f'state dimension: {sys.nx}')
import pprint
pprint.pprint(sys.stats)

# Defining an ODE based dynamical system in PSL

Consider the hamiltonian system on a plane defined by the ordinary differential equations: 
\dot{\theta} = \phi, \dot{\phi} = -\frac{g}{l}\sin\theta

$$x_0^{\prime} = x_1$$
$$x_1^{\prime} = \frac{g}{l} \sin(x_0)$$

Let's define using PSL with $$g = -1, l = 1$$ to simulate an undamped pendulum with no driving force.

In [ ]:
from neuromancer.psl.base import ODE_Autonomous, cast_backend
import numpy as np

class Hamiltonian(ODE_Autonomous):

    @property
    def params(self):
        variables = {'x0': [-2., 2.]} # Initial condition
        constants = {'ts': 0.1} # Time step for numerical simulation
        parameters = {'g': -1., # parameters of the hamiltonian system
                      'l': 1.}
        meta = {}
        return variables, constants, parameters, meta
        
    @cast_backend
    def equations(self, t, x):
        return [x[1], (self.g/self.l)*np.sin(x[0])]
    

In [ ]:
sys = Hamiltonian()
print(f'Changing inputs and states: {sys.variables}')
print(f'Constants defining simulation: {sys.constants}')
print(f'Parameters of the system: {sys._params}')
print(f'state dimension: {sys.nx}')
import pprint
pprint.pprint(sys.stats)
sys.show()
plt.close()

In [ ]:
def plot_field(sys, xlim, ylim, scale=100, density=15j):
    X, Y = np.mgrid[xlim[0]:xlim[1]:density, ylim[0]:ylim[1]:density]
    print(X.shape, Y.shape)
    u, v = sys.equations(None, [X, Y])
    print(u.shape, v.shape)
    plt.quiver(X, Y, u, v, color='b', scale=scale)


In [ ]:
plot_field(Hamiltonian(), (-np.pi, np.pi), (-5, 5))
sys.ts = 0.1
data1 = sys.simulate(nsim=32, x0=np.array([0, 1]))
data2 = sys.simulate(nsim=32, x0=np.array([-np.pi, 1.5]))
plt.plot(data1['X'][:, 0], data1['X'][:, 1], c='r')
plt.plot(data2['X'][:, 0], data2['X'][:, 1], c='g')
# plt.xlim((-2*np.pi, 2*np.pi))

# Modeling a dynamical system from data with NODEs

Let's say we have measurements of this system from a sample of initial conditions simulated for 400 time-steps. How many uniformly sampled initial conditions does it take to learn a decent model of this system? 

In [ ]:
from neuromancer.system import Node, System
import torch
# Sample 50 initial conditions covering the range of system behavior. 
x0s = np.random.uniform(low=np.array([-3*np.pi, -5.]), high=np.array([3*np.pi, 5]), size=(50, 2))
plot_field(sys, (-3*np.pi, 3*np.pi), (-5, 5))
plt.scatter(x0s[:, 0], x0s[:, 1], c='r')


In [ ]:
samples = [sys.simulate(nsim=96, x0=x0) for x0 in x0s]
X = np.concatenate([s['X'] for s in samples])
maxx, minx = X[:, 0].max(), X[:, 0].min()
maxy, miny = X[:, 1].max(), X[:, 1].min()

plot_field(sys, (minx, maxx), (miny, maxy))
for s in samples:
    plt.plot(s['X'][:, 0], s['X'][:, 1])
plt.scatter(x0s[:, 0], x0s[:, 1], c='r')


In [ ]:
# Change the default system stats to statistics from our training set
pprint.pprint(sys.stats)
sys.set_stats(sim={'X': X})
pprint.pprint(sys.stats)

# Create a pytorch dataloader for use in training the system model
samples = {'X': torch.stack([torch.tensor(s['X'], dtype=torch.float32) for s in samples])}
samples = sys.normalize(samples)
samples['X'].shape, type(samples['X'])


In [ ]:
from neuromancer.dataset import DictDataset

# Make a function to generate data
def get_data(sys, xmin, xmax, nstep, nsamples, name='train'):
    
    # Sample initial conditions
    x0s = np.random.uniform(low=xmin, high=xmax, size=(nsamples, sys.nx))
    
    # Create nsamples simulations with sampled initial conditions
    samples = [sys.simulate(nsim=nstep, x0=x0) for x0 in x0s]
    
    # Calculate min and max for plotting axis bounds    
    X = np.concatenate([s['X'] for s in samples])
    maxx, minx = X[:, 0].max(), X[:, 0].min()
    maxy, miny = X[:, 1].max(), X[:, 1].min()

    # Plot vector field, initial conditions and samples
    plot_field(sys, (minx, maxx), (miny, maxy))
    for s in samples:
        plt.plot(s['X'][:, 0], s['X'][:, 1])
    plt.scatter(x0s[:, 0], x0s[:, 1], c='r')
    
    if name == 'train':
        # Change the default system stats to statistics from our training set
        pprint.pprint(sys.stats)
        sys.set_stats(sim={'X': X})
        pprint.pprint(sys.stats)

    # Create a pytorch dataloader
    samples = {'X': torch.stack([torch.tensor(s['X'], dtype=torch.float32) for s in samples])}
    
    samples = sys.normalize(samples)
    samples['xn'] = samples['X'][:, 0:1, :]
    data = DictDataset(samples, name=name)
    loader = torch.utils.data.DataLoader(data, num_workers=1, batch_size=50, collate_fn=data.collate_fn,
                                         shuffle=True)
    return loader

In [2]:
train_data = get_data(sys, np.array([-3*np.pi, -5.]), np.array([3*np.pi, 5]), 96, 50, name='train')

NameError: name 'get_data' is not defined

In [3]:
dev_data = get_data(sys, np.array([-3*np.pi, -5.]), np.array([3*np.pi, 5]), 96, 50, name='dev')

NameError: name 'get_data' is not defined

In [4]:
from neuromancer.system import Node
from neuromancer.modules.blocks import MLP
from neuromancer.dynamics.integrators import Euler
from neuromancer.modules.activations import SoftExponential

# linargs = {'sigma_min': .95, 'sigma_max': 1.05}
class MultipleShootingEulerIntegrator(torch.nn.Module):
    """
    Simple black-box NODE
    """
    def __init__(self, nx, hsize, nlayers, ts):
        super().__init__()
        self.dx = MLP(nx, nx, hsizes=[hsize for h in range(nlayers)], nonlin=SoftExponential)
        self.integrator = Euler(self.dx, h=torch.tensor(ts))

    def forward(self, x1, xn):
        """

        :param xn: (Tensor, shape=(batchsize, nx)) State
        :param u: (Tensor, shape=(batchsize, nu)) Control action
        :return: (Tensor, shape=(batchsize, nx)) xn+1
        """
        return self.integrator(x1), self.integrator(xn)

nx = 2
hsize = 64
nlayers = 3
integrator = MultipleShootingEulerIntegrator(nx, hsize, nlayers, sys.ts)
node = Node(integrator, ['X', 'xn'], ['xstep', 'xn'])
system = System([node])
system.show()

ModuleNotFoundError: No module named 'functorch'

In [ ]:
from neuromancer.trainer import Trainer
from neuromancer.loggers import BasicLogger
from neuromancer.constraint import variable
from neuromancer.loss import PenaltyLoss
from neuromancer.problem import Problem

logger = BasicLogger(stdout=['train_objective_loss', 'dev_loss1', 'dev_loss2'], verbosity=1)
opt = torch.optim.AdamW(integrator.parameters(), 0.01)
xpred = variable('xn')[:, :-1, :]
xtrue = variable('X')
x1pred = variable('xstep')

loss1 = 0.1*(xpred == xtrue) ^ 2
loss1.update_name('loss1')
loss2 = (x1pred[:, :-1, :] == xtrue[:, 1:, :])^2
loss2.update_name('loss2')

obj = PenaltyLoss([loss1, loss2], [])
problem = Problem([system], obj)

trainer = Trainer(problem, train_data, dev_data, dev_data, opt, logger,
                  epochs=200,
                  patience=1000,
                  train_metric='train_objective_loss',
                  dev_metric='dev_loss',
                  test_metric='test_loss',
                  eval_metric='dev_loss')


In [ ]:
best_model = trainer.train()

In [ ]:
class rhs_wrapper:
    
    def __init__(self, dx, sys):
        self.dx = dx
        self.sys = sys
        
    def equations(self, t, X):
        X = torch.tensor(np.stack(X, axis=-1), dtype=torch.float32).reshape(-1, 2)
        print(X.shape)
        X = self.sys.normalize(X, key='X')
        output = self.dx(X)
        output = self.sys.denormalize(output, key='X')
        output = output.reshape(15, 15, 2).detach().numpy()
        return output[:, :, 0], output[:, :, 1]

problem.load_state_dict(best_model)
        
rhs = integrator.dx
func = rhs_wrapper(rhs, sys)
plot_field(func, (-3*np.pi, 3*np.pi), (-5, 5))

In [ ]:
plot_field(sys, (-3*np.pi, 3*np.pi), (-5, 5))